**This function is used to extract tbe label value of the string.**

As all the inputs are nested links in wiki data so its necessary to extaract the label using Web scraping.
Used BeautifulSoup

In [64]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

import pprint
import requests
# Defined a function to convert links to labels to put in text 
def ConverterFxn(input_Values):
  input_Values = input_Values.replace('entity','wiki')
  html = urlopen(input_Values)
  bsh = BeautifulSoup(html.read(), 'html.parser')
  x = bsh.h1
  x = str(x)
  # Converting link to usable form as extracted links are nested <Label type> and cant be used to web scrape
  Lidx = x.find('</span>')
  Iidx = x.find('wikibase-title-label') + 22
  New_string = ""
  for i in range(Iidx,Lidx):
    New_string = New_string + x[i]
  return(New_string)

In [65]:
import pprint
from pydoc import describe
from re import S
import requests
# To Get Search Result we search the query o wiki data search box utilise the most relevant search
API_ENDPOINT = "https://www.wikidata.org/w/api.php"
query = input('Search Here ')

params = {
    'action': 'wbsearchentities',
    'format': 'json',
    'language': 'en',
    'search': query
}

r = requests.get(API_ENDPOINT, params = params)
Data = r.json()['search'][0]
print(r.json()['search'][0])

Search Here Luknow
{'id': 'Q47916', 'title': 'Q47916', 'pageid': 49956, 'display': {'label': {'value': 'Lucknow', 'language': 'en'}, 'description': {'value': 'capital of Uttar Pradesh, India', 'language': 'en'}}, 'repository': 'wikidata', 'url': '//www.wikidata.org/wiki/Q47916', 'concepturi': 'http://www.wikidata.org/entity/Q47916', 'label': 'Lucknow', 'description': 'capital of Uttar Pradesh, India', 'match': {'type': 'label', 'language': 'ug', 'text': 'Luknow'}, 'aliases': ['Luknow']}


**Checking if the Data (initial) is working**

In [66]:
# Checking the functionality of link of page from which we will extracct data
print(Data['concepturi'])

http://www.wikidata.org/entity/Q47916


Installing qwikidata api

In [67]:
!pip3 install qwikidata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Installing mkwikidata

In [68]:
!pip3 install mkwikidata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Code to make/edit optimised queries for different links and instances**

In [69]:
import pprint
import mkwikidata

import requests

query2 = '''
  { ?article  schema:about       ?item ;
              schema:inLanguage  "en" ;
              schema:isPartOf    <https://en.wikipedia.org/>
              '''              
query3 = f'''          
    FILTER ( ?item = <{Data['concepturi']}> )
    '''
query4 = ''' 
   OPTIONAL
      { ?item  wdt:P1448  ?Name }
   OPTIONAL
      { ?item  wdt:P361  ?City_Part_Of}
   OPTIONAL
      { ?item  wdt:P625  ?Coordinate_Location }
   OPTIONAL
      { ?item  wdt:P1540  ?Male_Population }
   OPTIONAL
      { ?item  wdt:P1539  ?Female_Population }
   OPTIONAL
      { ?item  wdt:P6343  ?City_Urban_Population }
   OPTIONAL
      { ?item  wdt:P2044  ?SeaLevel }
   OPTIONAL
      { ?item  wdt:P421  ?TimeZone }
   OPTIONAL
      { ?item  wdt:P2046  ?City_Area}
   OPTIONAL
      { ?item  wdt:P6499  ?Place_Literacy}
   OPTIONAL
      { ?item  wdt:P6498  ?Place_Illetracy}
   OPTIONAL
      { ?item  wdt:P10091  ?City_Death_Rate}
   OPTIONAL
      { ?item  wdt:P8763  ?City_Birth_Rate}
   OPTIONAL
      { ?item  wdt:P138  ?City_named_After}
   OPTIONAL
      { ?item  wdt:P3529  ?Place_MedianIncome}
   OPTIONAL
      { ?item  wdt:P281  ?City_postal_Code}
   
    SERVICE wikibase:label
      { bd:serviceParam
                  wikibase:language  "en"
      }
  }
GROUP BY ?item ?itemLabel ?itemDescription
'''


url = 'https://query.wikidata.org/sparql'
query1 = f'''
PREFIX  schema: <http://schema.org/>
PREFIX  bd:   <http://www.bigdata.com/rdf#>
PREFIX  wdt:  <http://www.wikidata.org/prop/direct/>
PREFIX  wikibase: <http://wikiba.se/ontology#>

SELECT DISTINCT  ?item ?itemLabel ?itemDescription (SAMPLE(?Name) AS ?NameSample)  (SAMPLE(?City_Part_Of) AS ?City_Part_OfSample) (SAMPLE(?Coordinate_Location) AS ?Coordinate_LocationSample) (SAMPLE(?Male_Population) AS ?Male_PopulationSample) (SAMPLE(?Female_Population) AS ?Female_PopulationSample) (SAMPLE(?City_Urban_Population) AS ?City_Urban_PopulationSample) (SAMPLE(?SeaLevel) AS ?SeaLevelSample) (SAMPLE(?TimeZone) AS ?TimeZone) (SAMPLE(?City_Area) AS ?City_AreaSample) (SAMPLE(?Place_Literacy) AS ?Place_LiteracySample) (SAMPLE(?Place_Illetracy) AS ?Place_IlletracySample) (SAMPLE(?City_Birth_Rate) AS ?City_Birth_RateSample) (SAMPLE(?City_Death_Rate) AS ?City_Death_RateSample) (SAMPLE(?City_named_After) AS ?City_named_AfterSample) (SAMPLE(?Place_MedianIncome) AS ?Place_MedianIncomeSample) (SAMPLE(?City_postal_Code) AS ?City_postal_CodeSample)

WHERE

'''
query = query1 + query2 + query3 + query4


r = requests.get(url, params = {'format': 'json', 'query': query})
data = r.json()


**Checking if generated query is in perfect form**

In [70]:
print(query)


PREFIX  schema: <http://schema.org/>
PREFIX  bd:   <http://www.bigdata.com/rdf#>
PREFIX  wdt:  <http://www.wikidata.org/prop/direct/>
PREFIX  wikibase: <http://wikiba.se/ontology#>

SELECT DISTINCT  ?item ?itemLabel ?itemDescription (SAMPLE(?Name) AS ?NameSample)  (SAMPLE(?City_Part_Of) AS ?City_Part_OfSample) (SAMPLE(?Coordinate_Location) AS ?Coordinate_LocationSample) (SAMPLE(?Male_Population) AS ?Male_PopulationSample) (SAMPLE(?Female_Population) AS ?Female_PopulationSample) (SAMPLE(?City_Urban_Population) AS ?City_Urban_PopulationSample) (SAMPLE(?SeaLevel) AS ?SeaLevelSample) (SAMPLE(?TimeZone) AS ?TimeZone) (SAMPLE(?City_Area) AS ?City_AreaSample) (SAMPLE(?Place_Literacy) AS ?Place_LiteracySample) (SAMPLE(?Place_Illetracy) AS ?Place_IlletracySample) (SAMPLE(?City_Birth_Rate) AS ?City_Birth_RateSample) (SAMPLE(?City_Death_Rate) AS ?City_Death_RateSample) (SAMPLE(?City_named_After) AS ?City_named_AfterSample) (SAMPLE(?Place_MedianIncome) AS ?Place_MedianIncomeSample) (SAMPLE(?City_

**Checking extracted data extracted with help of the query**

In [71]:
pprint.pprint(data)

{'head': {'vars': ['item',
                   'itemLabel',
                   'itemDescription',
                   'NameSample',
                   'City_Part_OfSample',
                   'Coordinate_LocationSample',
                   'Male_PopulationSample',
                   'Female_PopulationSample',
                   'City_Urban_PopulationSample',
                   'SeaLevelSample',
                   'TimeZone',
                   'City_AreaSample',
                   'Place_LiteracySample',
                   'Place_IlletracySample',
                   'City_Birth_RateSample',
                   'City_Death_RateSample',
                   'City_named_AfterSample',
                   'Place_MedianIncomeSample',
                   'City_postal_CodeSample']},
 'results': {'bindings': [{'City_AreaSample': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal',
                                               'type': 'literal',
                                               'valu

**Checking nested dictionary's functionality**

In [72]:
print(data['head']['vars'][8])

City_Urban_PopulationSample


In [73]:
# print(data['results']['bindings'][0]['NameSample']['value'])

**Converting data to usable form: **

1.   Convertion of extracted data to usable variables.
2.   Converting the data links to labels that could be used in text
1.   Making sentences for the newly made variables.
2.   printing of the variables 

Note : If any of the existing steps fails the variable will be skipped hence either the data is not present or non-editable.

*All the variables that were required in all the groups are pre-labeled.*

In [74]:
City = data['results']['bindings'][0]['itemLabel']['value']
output = ''

try:
  Description = data['results']['bindings'][0]['itemDescription']['value']
  Place_Description = f''' {City} is {Description}.'''
  output = output + Place_Description 
  print(Place_Description)
except:
  pass  

try:
  City_named_After = data['results']['bindings'][0]['City_named_AfterSample']['value']
  City_named_After = ConverterFxn(City_named_After)
  Named_After = f''' {City} is named after {City_named_After}.'''
  output = output + Named_After
  print(Named_After)
except:
  pass  


try:
  City_Part_Of = data['results']['bindings'][0]['City_Part_OfSample']['value']
  City_Part_Of = ConverterFxn(City_Part_Of)
  Part_Of = f''' It is part of {City_Part_Of}.'''
  output = output + Part_Of
  print(Part_Of)
except:
  pass

try:
  City_postal_Code = data['results']['bindings'][0]['City_postal_CodeSample']['value']
  Place_Postal_Code = f''' The place have postal code {City_postal_Code}.'''
  output = output + Place_Postal_Code
  print(Place_Postal_Code)
except:
  pass

try:
  Coordinate_Location = data['results']['bindings'][0]['Coordinate_LocationSample']['value']
  Place_location = f''' {City} has its coordinate location as {Coordinate_Location}.'''
  output = output + Place_location
  print(Place_location)
except:
  pass

try:
  City_Area = data['results']['bindings'][0]['City_AreaSample']['value']
  Area = f''' It has its {City_Area} km square area.'''
  output = output + Area
  print(Area)
except:
  pass

try:
  Male_Population = data['results']['bindings'][0]['Male_PopulationSample']['value']
  Male_Population_ = f''' It has its highest recorded male population as {Male_Population}.'''
  output = output + Male_Population_
  print(Male_Population_)
except:
  pass

try:
  Female_Population = data['results']['bindings'][0]['Female_PopulationSample']['value']
  Female_Population_ = f''' It has its highest recorded female population as {Female_Population}.'''
  output = output + Female_Population_
  print(Female_Population_)
except:
  pass

try:
  City_Urban_Population = data['results']['bindings'][0]['City_Urban_PopulationSample']['value']
  Urban_Population = f''' Its present urban population is {City_Urban_Population}.'''
  output = output + Urban_Population
  print(Urban_Population)
except:
  pass

try:
  SeaLevel = data['results']['bindings'][0]['SeaLevelSample']['value']
  Place_elevation_Above_Sea_Level = f''' Its elevation above sea level is {SeaLevel}.'''
  output = output + Place_elevation_Above_Sea_Level
  print(Place_elevation_Above_Sea_Level)
except:
  pass

try:
  TimeZone = data['results']['bindings'][0]['TimeZone']['value']
  TimeZone = ConverterFxn(TimeZone)
  Place_TimeZone = f''' It is located in timezone {TimeZone}.'''
  output = output + Place_TimeZone
  print(Place_TimeZone)
except:
  pass


try:
  Place_Literacy = data['results']['bindings'][0]['Place_LiteracySample']['value']
  Place_Illiteracy = NetWorth = data['results']['bindings'][0]['Place_IlletracySample']['value']
  Place_Literacy = f''' The location got approximately {Place_Literacy} no of literate people and {Place_Illiteracy} no of illiterates.'''
  output = output + Place_Literacy
  print(Place_Literacy)
except:
  pass

try:
  City_Birth_Rate = NetWorth = data['results']['bindings'][0]['City_Birth_RateSample']['value']
  Place_Birth_Rate = f''' {City} has a BirthRate of {City_Birth_Rate}.'''
  output = output + Place_Birth_Rate
  print(Place_Birth_Rate)
except:
  pass

try:
  City_Death_Rate = NetWorth = data['results']['bindings'][0]['City_Death_RateSample']['value']
  Place_Death_Rate = f''' It has a death rate of {City_Death_Rate}.'''
  output = output + Place_Death_Rate
  print(Place_Death_Rate)
except:
  pass

try:
  Place_MedianIncome = NetWorth = data['results']['bindings'][0]['Place_MedianIncomeSample']['value']
  Place_MedianIncome_ = f''' {City} has Median Income is {Place_MedianIncome}.'''
  output = output + Place_MedianIncome_
  print(Place_MedianIncome_)
except:
  pass




 Lucknow is capital of Uttar Pradesh, India.
 The place have postal code 226001–226026.
 Lucknow has its coordinate location as Point(80.947 26.847).
 It has its 631 km square area.
 It has its highest recorded male population as 1470133.
 It has its highest recorded female population as 1345468.
 Its elevation above sea level is 123.
 It is located in timezone UTC+05:30.


In [75]:
pprint.pprint(output)

(' Lucknow is capital of Uttar Pradesh, India. The place have postal code '
 '226001–226026. Lucknow has its coordinate location as Point(80.947 26.847). '
 'It has its 631 km square area. It has its highest recorded male population '
 'as 1470133. It has its highest recorded female population as 1345468. Its '
 'elevation above sea level is 123. It is located in timezone UTC+05:30.')
